# Join PySpark dataframes

- [Merging guide](https://www.cojolt.io/blog/joining-merging-data-with-pyspark-a-complete-guide)

In [0]:
# Let's import PySpark and initialize the Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [0]:
# Let's create 2 df, so we can merge them later
from datetime import date
from pyspark.sql import Row

# Horizontal/wide df
df1 = spark.createDataFrame([
    Row(SUBJID='1', AGE=25, SEX='F',  HEIGHT= 150, SCREENED=date(2024, 1, 15)),
    Row(SUBJID='2', AGE=56, SEX='M',  HEIGHT=168, SCREENED=date(2024, 2, 10)),
    Row(SUBJID='3', AGE=44, SEX=None, HEIGHT=170, SCREENED=date(2024, 1, 17))
])

# Vertical/narrow df
df2 =  spark.createDataFrame([
    Row(SUBJID='1', PARAM='Param1', VALUE='12'   ),
    Row(SUBJID='1', PARAM='Param2', VALUE='Hello'),
    Row(SUBJID='1', PARAM='Param3', VALUE=True   ),
    Row(SUBJID='3', PARAM='Param1', VALUE='14'   ),
    Row(SUBJID='3', PARAM='Param2', VALUE='Bye'),
    Row(SUBJID='3', PARAM='Param3', VALUE=False   ),
])

In [0]:
# Let's do a left join, using SUBJID as key for merging
merged_df = df2.join(df1, df1.SUBJID == df2.SUBJID, "left")

display(merged_df)

In [0]:
# Merging PySpark DataFrames can be achieved through multiple techniques. Below are different methods and their scenarios.

# 1. Inner Join (Default)
    # Selects records that have matching values in both DataFrames.
joined_df = df1.join(df2, df1.SUBJID == df2.SUBJID, "inner")

# 2. Left Join
    # Selects all records from the left DataFrame, and the matched records from the right DataFrame. 
    # The result is NULL from the right side if there is no match.
left_joined_df = df1.join(df2, df1.SUBJID == df2.SUBJID, "left")

# 3. Right Join
    # Selects all records from the right DataFrame, and the matched records from the left DataFrame. 
    # The result is NULL from the left side when there is no match.
right_joined_df = df1.join(df2, df1.SUBJID == df2.SUBJID, "right")

# 4. Full Outer Join
    # Returns records when there is a match in one of the DataFrames.
outer_joined_df = df1.join(df2, df1.SUBJID == df2.SUBJID, "outer")

# 5. Cross Join
    # Returns the Cartesian product of both DataFrames.
cross_joined_df = df1.crossJoin(df2)

# 6. Using SQL for joining
df1.createOrReplaceTempView("table1")
df2.createOrReplaceTempView("table2")

sql_query = 'SELECT * FROM table1 t1 JOIN table2 t2 ON t1.SUBJID = t2.SUBJID'
sql_joined_df = spark.sql(sql_query)

# 7. Broadcast Join
    # Broadcasting in PySpark is a way to optimize joins when one of the DataFrames is small enough to fit in memory.
    # By broadcasting the smaller DataFrame, we can avoid shuffling large amounts of data across the cluster.
from pyspark.sql.functions import broadcast
broadcast_joined_df = df1.join(broadcast(df2), df1.SUBJID == df2.SUBJID, "inner")

# 8. Natural Join
    # Automatic match on same column names, use with caution
natural_joined_df = df1.join(df2)

In [0]:
df = left_joined_df
df.display()

In [0]:
# After the merge, the BY columns are not coalesced into one column! This is different from Pandas behavior. See the column names, we have 2 SUBJID columns!
df.columns

# PySpark maintains metadata internally to manage column names and references. However, there is no direct API to explicitly retrieve the origin of each column in the merged DataFrame.

In [0]:
# If we wanted to use 'SUBJID', we would have to quality if (eg. df1['SUBJID']. Instead we could get rid of the extra SUBJID columns, the one coming from the df2 dataframe:
df = df.drop(df2['SUBJID'])
print(df.columns)

# Only one SUBJID remains. We don't need anymore to qualify it when using it 